## Getting Started

In [1]:
from arcgis.gis import GIS
from arcgis.geoenrichment import enrich
from arcgis.geoenrichment import create_report
from arcgis.geometry import Envelope
from arcgis.features.manage_data import generate_tessellation
from arcgis.features import FeatureSet
from getpass import getpass

In [2]:
password=getpass()
gis = GIS(username="YOUR-USERNAME", password=password)

········


## Build Analysis Areas

In [3]:
inland_empire_map = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map.basemap = 'openstreetmap'
inland_empire_map

MapView(layout=Layout(height='400px', width='100%'))

In [4]:
hexagons = generate_tessellation(None, bin_size=1, bin_size_unit='SquareMiles', bin_type='HEXAGON', context={"extent":inland_empire_map.extent})
inland_empire_map.add_layer(hexagons)

len(hexagons.layer.featureSet.features)

176

In [5]:
geometry_areas = [{"geometry":f["geometry"].copy()} for f in hexagons.layer.featureSet.features]

for area in geometry_areas:
    area["geometry"]["spatialReference"]={"wkid":3857}
    
geometry_areas[0]

{'geometry': {'rings': [[[-13062306.6791, 4028583.2685000002],
    [-13062906.5407, 4027535.6648999974],
    [-13064109.4833, 4027533.6699],
    [-13064712.784400001, 4028579.277800001],
    [-13064113.0335, 4029627.0714000016],
    [-13062909.8707, 4029629.0671000034],
    [-13062306.6791, 4028583.2685000002]]],
  'spatialReference': {'wkid': 3857}}}

In [6]:
enriched_areas=enrich(study_areas=geometry_areas, analysis_variables=["KeyGlobalFacts.TOTPOP","AtRisk.SENIORS_CY"])
enriched_areas.head()

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE
0,0,1,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,2901,209,"{""rings"": [[[-117.34069735692039, 33.998370815..."
1,1,2,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,1109,94,"{""rings"": [[[-117.3244995205204, 34.0061859289..."
2,2,3,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,2,0,"{""rings"": [[[-117.30827571292721, 33.998408265..."
3,3,4,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,1640,316,"{""rings"": [[[-117.29207489771376, 34.006219103..."
4,4,5,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,32,6,"{""rings"": [[[-117.27585404467956, 33.998437158..."


In [7]:
enriched_areas["SENIORS_PERCENT"] = enriched_areas["SENIORS_CY"]  / enriched_areas["TOTPOP"] * 100

senior_populated_area = enriched_areas[enriched_areas["SENIORS_PERCENT"] > 25]
senior_populated_area.shape[0]

9

In [8]:
inland_empire_map2 = gis.map('Redlands, CA, USA', zoomlevel=12)
inland_empire_map2.basemap = 'openstreetmap'

inland_empire_map2.add_layer(FeatureSet.from_dataframe(senior_populated_area.copy()))
inland_empire_map2

MapView(layout=Layout(height='400px', width='100%'))

In [9]:
inland_empire_map2.add_layer(FeatureSet.from_dataframe(enriched_areas), {'renderer':'ClassedSizeRenderer','field_name':'SENIORS_PERCENT', 'opacity':0.5})

In [10]:
most_senior_area = enriched_areas[enriched_areas.SENIORS_PERCENT == enriched_areas.SENIORS_PERCENT.max()]
most_senior_area

,ID,OBJECTID,sourceCountry,aggregationMethod,populationToPolygonSizeRating,apportionmentConfidence,HasData,TOTPOP,SENIORS_CY,SHAPE,SENIORS_PERCENT
53,53,54,US,BlockApportionment:US.BlockGroups,2.191,2.576,1,2197,779,"{""rings"": [[[-117.19480539330945, 34.037448478...",35.457442


In [11]:
fill_symbol = {
  "type": "esriSFS",
  "style": "esriSFSSolid",
  "color": [255,0,0,255],
    "outline": {
     "type": "esriSLS",
     "style": "esriSLSSolid",
     "color": [0,0,0,255],
     "width": 1
    }
}

inland_empire_map2.draw(most_senior_area.iloc[0].SHAPE, symbol = fill_symbol)

## Building Report

In [12]:
most_senior_study_area = {"geometry":most_senior_area.iloc[0].SHAPE}

# coronavirus impact planning report
report = create_report(study_areas=[most_senior_study_area], report={"itemid":"ac36af7d86b74071a1cff5dc11f52ae8"}, out_name="COVID Impact Planning Profile.pdf", out_folder="output")

In [13]:
# health care & insurace 
report = create_report(study_areas=[most_senior_study_area], report={"itemid":"d29fcd15f66f46edad86499675b0ec03"}, export_format='html', out_name="Health Care and Insurance.html", out_folder="output")